In [1]:
"""Python WPS execute"""
from owslib.wps import WebProcessingService, monitorExecution, printInputOutput
from os import system

In [2]:
#import owslib
#owslib.__version__ # version must be >=0.10.3

In [3]:
wps_url = "http://localhost:8093/wps"
#wps_url = "http://birdhouse-lsce.extra.cea.fr:8093/wps"
wps = WebProcessingService(url=wps_url, verbose=False)

In [4]:
print wps.identification.title

Flyingpigeon


In [5]:
for process in wps.processes:
    print '%s : \t %s' % (process.identifier, process.abstract)

visualisation : 	 Just testing a nice script to visualise some variables
segetalflora : 	 Species biodiversity of segetal flora. Imput files: variable:tas , domain: EUR-11 or EUR-44
indices_single : 	 This process calculates climate indices based on one single variable.
subset_countries : 	 This process returns only the given polygon from input netCDF files.
eobs_to_cordex : 	 downloads EOBS data in adaped CORDEX format
ensembleRobustness : 	 Calculates the robustness as the ratio of noise to signal in an ensemle of timeseries
analogs : 	 Search for days with analog pressure pattern
fetch : 	 This process downloads resources (limited to 50GB) to the local file system and returns a textfile with appropriate pathes


In [6]:
p = wps.describeprocess(identifier='ensembleRobustness')
for input in p.dataInputs:
    printInputOutput(input)
    print '\n'

 identifier=resource, title=NetCDF Files, abstract=NetCDF Files, data type=ComplexData
 Supported Value: mimeType=application/x-netcdf, encoding=None, schema=None
 Default Value: mimeType=application/x-netcdf, encoding=None, schema=None 
 minOccurs=1, maxOccurs=100


 identifier=start, title=Start Year, abstract=Beginn of the analysed period (e.g 1971; if not set, the first consistend year of the ensemble will be taken), data type=//www.w3.org/TR/xmlschema-2/#integer
 Any value allowed
 Default Value: None 
 minOccurs=0, maxOccurs=1


 identifier=end, title=End Year, abstract=End of the analysed period (e.g. 2050 if not set, the last consistend year of the ensemble will be taken), data type=//www.w3.org/TR/xmlschema-2/#integer
 Any value allowed
 Default Value: None 
 minOccurs=0, maxOccurs=1


 identifier=timeslice, title=Time slice, abstract=Time slice (in years) for robustness reference (default=10)), data type=//www.w3.org/TR/xmlschema-2/#integer
 Any value allowed
 Default Value: 

In [10]:
path = 'file:///homel/nhempel/data/cache/pywps/'

files = ['tas_EUR-44_NOAA-GFDL-GFDL-ESM2M_historical_r1i1p1_SMHI-RCA4_v1_sem_198012-199011.nc',
'tas_EUR-44_NOAA-GFDL-GFDL-ESM2M_historical_r1i1p1_SMHI-RCA4_v1_sem_199012-200011.nc',
'tas_EUR-44_NOAA-GFDL-GFDL-ESM2M_historical_r1i1p1_SMHI-RCA4_v1_sem_200012-200511.nc',
'tas_EUR-44_NOAA-GFDL-GFDL-ESM2M_rcp45_r1i1p1_SMHI-RCA4_v1_sem_200512-201011.nc',
'tas_EUR-44_NOAA-GFDL-GFDL-ESM2M_rcp45_r1i1p1_SMHI-RCA4_v1_sem_201012-202011.nc',
'tas_EUR-44_NOAA-GFDL-GFDL-ESM2M_rcp45_r1i1p1_SMHI-RCA4_v1_sem_202012-203011.nc',
'tas_EUR-44_NOAA-GFDL-GFDL-ESM2M_rcp45_r1i1p1_SMHI-RCA4_v1_sem_203012-204011.nc'   
]

In [12]:
from os.path import join

execute = wps.execute(
    identifier="ensembleRobustness", #indices_clipping",
    inputs=[
       ("resource",path+files[0]),
       ("resource",path+files[1]),
       ("resource",path+files[2]),
        ("resource",path+files[3]),
        ("resource",path+files[4]),
        ("resource",path+files[5]),
        ("start",'1991'),
        ("end",'2005'),
       #("resource",path+files[3]),
       ("timeslice",'1'),
        ("variable",'tas')
           ])

monitorExecution(execute, sleepSecs=5)
print execute.getStatus()

for o in execute.processOutputs:
    print o.reference
    

ProcessSucceeded
http://localhost:8090/wpsoutputs/flyingpigeon/output_graphic-bedc7e18-d723-11e5-a1ca-3563193f887d.png
http://localhost:8090/wpsoutputs/flyingpigeon/output_high-bedc7e18-d723-11e5-a1ca-3563193f887d.nc
http://localhost:8090/wpsoutputs/flyingpigeon/output_text-bedc7e18-d723-11e5-a1ca-3563193f887d.txt
http://localhost:8090/wpsoutputs/flyingpigeon/output_low-bedc7e18-d723-11e5-a1ca-3563193f887d.nc
http://localhost:8090/wpsoutputs/flyingpigeon/output_signal-bedc7e18-d723-11e5-a1ca-3563193f887d.nc


In [13]:
graphic = 'http://localhost:8090/wpsoutputs/flyingpigeon/output_graphic-bedc7e18-d723-11e5-a1ca-3563193f887d.png'

from IPython.display import Image
from IPython.core.display import HTML 
Image(url= graphic )